## PyCUDA

PyCUDA is a Python package that provides a lean interface to CUDA API and CUDA C. It introduces a minimal amount of new concepts and thus is a good choice for programmers familiar with Python and CUDA. In this section we will review the major components of PyCUDA and how one can use them to write CUDA applications in Python.

**GitHub**: https://github.com/inducer/pycuda<br />
**Documentation**: https://documen.tician.de/pycuda

### Getting started

Let's begin by reviewing a PyCUDA application that multiplies elements of a floating-point array by a specified factor:

In [ ]:
# Import PyCUDA
# Common for many PyCUDA programms
import pycuda.autoinit
import pycuda.driver
import pycuda.compiler

import numpy

mod = pycuda.compiler.SourceModule("""
__global__  void  double_array (float* d_a, unsigned int n_elements, float factor) {
    unsigned int idx = threadIdx.x + blockIdx.x * blockDim.x;
    if (idx < n_elements)
        d_a[idx] *= factor;
}""")

if __name__ == '__main__':
    
    # Input parameters. Feel free to change them
    data_size = 1<<20 # 1048576
    factor = 10 * numpy.random.random()
    
    # Initialize array on host
    h_array = numpy.random.uniform(0, 50, size=data_size)
    
    # Convert to 32-bit floating-point
    h_array = h_array.astype(numpy.float32)
    
    # Allocate device memory
    d_array = pycuda.driver.mem_alloc(h_array.nbytes)
    
    # Copy host array to device memory
    pycuda.driver.memcpy_htod(d_array, h_array)
    
    # Find reference to the function
    multiplication_kernel = mod.get_function('double_array')
    
    # Set block and grid dimensions
    blockDim = (128, 1, 1)
    grid_size_x = (data_size + blockDim[0] - 1) // blockDim[0]
    gridDim = (grid_size_x, 1, 1)
    
    # Launch the kernel
    multiplication_kernel(d_array, numpy.uint32(data_size), numpy.float32(factor), grid=gridDim, block=blockDim)
    
    # Transfer the results back to host
    results = numpy.empty_like(h_array)
    pycuda.driver.memcpy_dtoh(results, d_array)
    
    # Release GPU memory
    d_array.free()
    
    # Make sure the results are correct
    assert numpy.allclose(results, h_array * factor), "Failed"
    
    # Report success
    print("Success")

This is great but even though we didn't have to specify such things as directions of memory copy and compute the size of transferred data, this isn't very exciting, because we still had to not only write a CUDA kernel in CUDA C, but also perform all the memory and data manipulation steps typical for a CUDA C program.

PyCUDA provides a few ways to simplify the above program.

In [ ]:
# Import PyCUDA
# Common for many PyCUDA programms
import pycuda.autoinit
import pycuda.driver
import pycuda.compiler

import numpy

mod = pycuda.compiler.SourceModule("""
__global__  void  double_array (float* d_a, unsigned int n_elements, float factor) {
    unsigned int idx = threadIdx.x + blockIdx.x * blockDim.x;
    if (idx < n_elements)
        d_a[idx] *= factor;
}""")

if __name__ == '__main__':
    
    # Input parameters. Feel free to change them
    data_size = 1<<20 # 1048576
    factor = 10 * numpy.random.random()
    
    # Initialize array on host
    h_array = numpy.random.uniform(0, 50, size=data_size)
    
    # Convert to 32-bit floating-point
    h_array = h_array.astype(numpy.float32)
    
    # Save a copy to verify the results
    h_array_copy = h_array.copy()

    # Find reference to the function
    multiplication_kernel = mod.get_function('double_array')

    # Set block and grid dimensions
    blockDim = (128, 1, 1)
    grid_size_x = (data_size + blockDim[0] - 1) // blockDim[0]
    gridDim = (grid_size_x, 1, 1)
      
    # Launch the kernel
    multiplication_kernel(pycuda.driver.InOut(h_array), numpy.uint32(data_size), numpy.float32(factor), grid=gridDim, block=blockDim)
    
    # Make sure the results are correct
    assert numpy.allclose(h_array, h_array_copy * factor), "Failed"
    
    # Report success
    print("Success")

That's a little bit better! Here we used `pycuda.driver.InOut` convenience function which tells PyCUDA to copy  provided array to the GPU, do the calculations, transfer the results back to the host and save them to the same array, essentially, overwriting input. Thanks to `pycuda.driver.InOut`, we did not have to allocate memory on the device, transfer the data from host to gpu and back, and we didn't have to deallocate the GPU array.

If, instead, we wrote a different kernel that had input and output separated like so:

```
mod = pycuda.compiler.SourceModule("""
__global__  void  double_array (float* input, float *output, unsigned int n_elements, float factor) {
    unsigned int idx = threadIdx.x + blockIdx.x * blockDim.x;
    if (idx < n_elements)
        output[idx] = input[idx] * factor;
}""")
```

we could then use another two conveniece functions: `pycuda.driver.In` and `pycuda.driver.Out` to send data to and from the GPU:

In [ ]:
#... 
    # Placeholder for the final result
    results = numpy.empty_like(h_array)

#...      
    # Launch the kernel
    multiplication_kernel(pycuda.driver.In(h_array), pycuda.driver.Out(results), numpy.uint32(data_size), numpy.float32(factor), grid=gridDim, block=blockDim)
    
    # Make sure the results are correct
    assert numpy.allclose(results, h_array * factor), "Failed"

Essentially, both `pycuda.driver.In` and `pycuda.driver.Out` save us from manually allocating memory on the device, transferring data from host to device and back and then deallocating memory! Very convenient!

### But wait, there's more!

For such applications PyCUDA provides another convenience interface - `pycuda.gpuarray.GPUArray`, which associates Numpy arrays with arrays on the device, handles transfers, and allows the programmer to express array operations with Numpy syntax. For example, the above program can be slimmed down to:

In [ ]:
import pycuda.driver
import pycuda.autoinit
import pycuda.gpuarray

import numpy

if __name__ == '__main__':
    
    # Input parameters. Feel free to change them
    data_size = 1<<20 # 1048576
    factor = 10 * numpy.random.random()
    
    # Initialize array on host
    h_array = numpy.random.uniform(0, 50, size=data_size)
    
    # Convert to 32-bit floating-point
    h_array = h_array.astype(numpy.float32)
    
    # Associate h_array with an array on a GPU and do the calculations
    d_gpuarray = pycuda.gpuarray.to_gpu(h_array)
    d_results = factor * d_gpuarray

    # Copy the result to CPU using .get() method
    results = d_results.get()
    
    # Make sure the results are correct
    assert numpy.allclose(results, h_array * factor), "Failed"
    
    # Report success
    print("Success")

## Element-wise operations

To do a bit more complex mathematical operations on arrays, we can use the `pycuda.cumath` module, which provides such functions as `sqrt`, `sin`, `cos`, `log`, `exp`, and so on:

In [ ]:
import pycuda.driver
import pycuda.autoinit
import pycuda.gpuarray

import numpy

if __name__ == '__main__':
    h_array = numpy.random.randn(4000,4000).astype(numpy.float32)
    d_gpuarray = pycuda.gpuarray.to_gpu(h_array)
    
    # Compute 
    result = pycuda.cumath.sqrt(pycuda.cumath.fabs(d_gpuarray)).get()
    assert numpy.allclose(numpy.sqrt(numpy.abs(h_array)), result), "Failed"
    print("Success")

PyCUDA also provides a module for custom element-wise operations, `pycuda.elementwise`:

In [ ]:
import pycuda.autoinit
import pycuda.gpuarray
import pycuda.elementwise
import pycuda.curandom

if __name__ == '__main__':
    
    # Generate random PyCUDA arrays on GPU
    a_gpu = pycuda.curandom.rand((50,))
    b_gpu = pycuda.curandom.rand((50,))

    # Set up an element-wise operation
    lin_comb = pycuda.elementwise.ElementwiseKernel(
            "float a, float *x, float b, float *y, float *z",
            "z[i] = a*x[i] + b*y[i]",
            "linear_combination")

    c_gpu = pycuda.gpuarray.empty_like(a_gpu)
    lin_comb(5, a_gpu, 6, b_gpu, c_gpu)
    
    print(c_gpu.get())

## Reductions

Finding a reduced value for an array on a GPU is a well-known pain point: as you remember, threads on a GPU device don't know much about each other unless they're in the same "block". PyCUDA provides several convenience methods that compute several reductions on GPU arrays: sum, dot product with another matrix, max, min, as well as versions of these reductions but applied to subsets of arrays.

In [ ]:
import pycuda.autoinit
import pycuda.gpuarray
import pycuda.curandom

if __name__ == '__main__':
    
    # Generate random PyCUDA arrays on GPU
    a_gpu = pycuda.curandom.rand((50,))
    b_gpu = pycuda.curandom.rand((50,))
    
    maximum = pycuda.gpuarray.max(a_gpu).get()
    dot_product = pycuda.gpuarray.dot(a_gpu, b_gpu).get()
    
    print("max(a)    = ", maximum)
    print("dot(a, b) = ", dot_product)

Lastly, similar to custom element-wise operations, PyCUDA provides a mechanism for custom reductions, though this topic is beyond the scope of this overview.

## References

1. PyCUDA documentation: https://documen.tician.de/pycuda